In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.__version__

'4.2.0'

In [3]:
df = pd.read_csv('data/food-box-data.csv', parse_dates=True)

df['date'] = pd.to_datetime(df['date'])

In [4]:
mask = df['final_method'] != 'reallocate'

Create a basic bar chart
- altair can handle the aggregation (`y='count()'`)
- optionally specify `:N` for nominal (categorical), `:O` for ordinal (ordered category) `:Q` for quantitative, or `:T` for temporal
- if you don't specify, Altair will guess the best default

In [5]:
alt.Chart(df[mask]).mark_bar().encode(
    x='date:T',
    y='count():Q'
).properties(title='Pickup and Delivery Counts', width=800)

alt.Chart(...)

By default the bars will be thin points in time when the axis is temporal. You can use a [TimeUnit Transform](https://altair-viz.github.io/user_guide/transform/timeunit.html) to "discretize" them into nominal or ordinal buckets

In [6]:
alt.Chart(df[mask]).mark_bar().encode(
    x=alt.X('monthdate(date):O', axis=alt.Axis(format='%b-%d')),
    y='count():Q'
).properties(title='Pickup and Delivery Counts', width=800)

alt.Chart(...)

### Stacked bar chart
To stack, you can just add the `color` property...

In [26]:
alt.Chart(df[mask]).mark_bar().encode(
    x=alt.X('monthdate(date):O', axis=alt.Axis(format='%b-%d')),
    y='count():Q',
    color='final_method'
).properties(title='Pickup and Delivery Counts', width=800)

alt.Chart(...)

### Grouped bar chart
To group, you can add the `column` and `color` properties. Think of the `column` as the groups, and `x` as the mini axis for each group. So usually, you will want to set `x` and `color` to the same data.

In [7]:
alt.Chart(df[mask]).mark_bar().encode(
    x='weekday:N',
    y='count():Q',
    column='monthdate(date):O',
    color='weekday:N'
).properties(title='Pickup and Delivery Counts')

alt.Chart(...)

Here are some ways to make it look a little cleaner.

In [37]:
alt.Chart(df[mask]).mark_bar().encode(
    x=alt.X('weekday:N', axis=alt.Axis(ticks=False, labels=False), title=''),
    y=alt.Y('count():Q', axis=alt.Axis(grid=False, title='Box Count')),
    column=alt.Column('monthdate(date):O', title='', spacing=15, header=alt.Header(orient='bottom')),
    color=alt.Color('weekday:N', title='')
).configure_view(
    stroke=None
).properties(
    title='Pickup and Delivery Counts'
)

alt.Chart(...)

`orient='bottom'` on the column header does not seem to be working and could be a bug in altair 4.2.0 based on some other people reporting the same problem.

As a workaround, we can explicitly set the height, and set `labelPadding` on the column header to the height + 20 or so

You can also set the font size with the `labelFontSize` property

In [40]:
alt.Chart(df[mask]).mark_bar().encode(
    x=alt.X('weekday:N', axis=alt.Axis(ticks=False, labels=False), title=''),
    y=alt.Y('count():Q', axis=alt.Axis(grid=False, title='Box Count')),
    column=alt.Column('monthdate(date):O', title='', spacing=15, header=alt.Header(orient='bottom', labelPadding=-320, labelFontSize=12)),
    color=alt.Color('weekday:N', title='')
).configure_view(stroke=None).properties(title='Pickup and Delivery Counts', height=300)

alt.Chart(...)

Now add a tooltip by passing the `tooltip` param to `encode`. It can be an array of strings representing the columns to show, or `alt.Tooltip` objects if you want to customize them further.

for example: `alt.Tooltip('mean(size):Q', title='boxes', format='.1f')]`

In [30]:
alt.Chart(df[mask]).mark_bar().encode(
    x=alt.X('weekday:N', axis=alt.Axis(ticks=False, labels=False), title=''),
    y=alt.Y('count():Q', axis=alt.Axis(grid=False, title='Box Count')),
    column=alt.Column('monthdate(date):O', title='', spacing=15, header=alt.Header(orient='bottom', labelPadding=-320)),
    color=alt.Color('weekday:N', title=''),
    tooltip=['date','weekday', alt.Tooltip('count():Q', title='boxes')]
).configure_view(stroke=None).properties(title='Pickup and Delivery Counts', height=300)

alt.Chart(...)

In [118]:
alt.Header?